### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [24]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):       
                # if the rating exists
                if ratings_mat[i, j] > 0:     
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix(user_mat & movie_mat) in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * 2 * diff * movie_mat[k, j]
                        movie_mat[k, j] += learning_rate * 2 * diff * user_mat[i, k]
                    
        # print results for iteration
        print(f'iteration {iteration + 1}: {sse_accum / num_ratings}')
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [25]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
iteration 1: 46.78635930145328
iteration 2: 17.39064072471842
iteration 3: 4.366839094172723
iteration 4: 2.6363440543690233
iteration 5: 2.4663331231115704
iteration 6: 2.3918445755125193
iteration 7: 2.3219253736681127
iteration 8: 2.2462440155376138
iteration 9: 2.162689851550438
iteration 10: 2.0710244515600236


In [26]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.13807536   9.14366998  10.35142905  10.15588122]
 [  8.81130355   6.92296973   8.96186626   9.11352748]
 [  7.77950502   6.92647062   8.80569015   7.3189518 ]
 [  9.58552559   8.16576227   9.59768241  10.01795034]
 [  8.54921023   6.88224283   9.51057638   8.38891217]
 [  6.95112859   5.40069616   8.08248611   6.40020668]
 [  9.12222797   8.22899044   9.56676551   9.23267892]
 [  8.39224488   7.11186159   8.08495552   8.5782696 ]
 [  8.45194204   7.73478518   8.95316344   8.28602336]
 [  7.75258171   6.7022366    8.38793796   7.3455275 ]
 [  8.90863973   7.84149471   9.4098616    8.71328002]
 [  9.50771351   8.47461181  10.10088919   9.52377897]
 [  9.39957709   8.098806     9.35672199   9.49652082]
 [  6.56898309   6.46812894   5.98092109   7.13716587]
 [  9.88605464   8.24891439  10.17506078   9.85113603]
 [  9.73167068   8.98330263  10.00945236   9.66102464]
 [  8.33981862   6.92377331   9.10144477   8.07800447]
 [  7.0945275    6.63507213   6.26580514   7.8483881 ]
 [  8.3559

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [27]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
iteration 1: 41.87502676850846
iteration 2: 13.860639160379142
iteration 3: 4.023891526288205
iteration 4: 2.8506331152790536
iteration 5: 2.7307436232538533
iteration 6: 2.700786863249967
iteration 7: 2.683957187861699
iteration 8: 2.6688528371176212
iteration 9: 2.6530197710119867
iteration 10: 2.6357866681018676
iteration 11: 2.616843962099815
iteration 12: 2.5959546484682905
iteration 13: 2.5729018264286903
iteration 14: 2.5474837004850377
iteration 15: 2.5195169660073757
iteration 16: 2.488840686951317
iteration 17: 2.4553187955367783
iteration 18: 2.418840283392334
iteration 19: 2.3793162545447384
iteration 20: 2.336673179489183
iteration 21: 2.2908421413466855
iteration 22: 2.2417446532927743
iteration 23: 2.1892767237774025
iteration 24: 2.1332941371441314
iteration 25: 2.0736031865886084
iteration 26: 2.0099620332429726
iteration 27: 1.9420980883419907
iteration 28: 1.8697459148837006
iteration 29: 1.7927077433501801
ite

In [28]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99997983   9.99998692   9.9999948   10.00002186]
 [  9.99999067   3.99999423   8.99999792  10.00001018]
 [  8.00001892   9.00001242  10.00000515   4.9999798 ]
 [  8.9999953    7.9999971    9.99999899  10.00000521]
 [ 10.00001262   5.00000834   9.00000349   8.99998654]
 [  6.00002049   4.00001314  10.00000517   5.99997793]
 [  8.99997911   7.9999863    9.99999442   9.00002256]
 [ 10.00000911   5.00000585   9.00000235   7.99999027]
 [  6.99997912   7.99998651   9.99999466   8.00002259]
 [  8.99998881   4.99999287   8.99999727   7.00001217]
 [  8.99999158   7.9999949    9.99999826   8.00000926]
 [  8.99999419   9.99999675   9.99999915   9.00000654]
 [ 10.00002066   9.00001373  10.00000583   7.999978  ]
 [  5.0000172    8.00001133   5.00000473   7.99998164]
 [ 10.00001059   8.00000679  10.00000271   9.9999887 ]
 [  9.00000517   9.00000313  10.00000113   9.99999451]
 [  8.99997551   7.99998397   7.9999935    8.00002646]
 [ 10.0000074    8.00000489   1.0000021    9.99999224]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [29]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [30]:
# run SVD on the matrix with the missing value
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
iteration 1: 45.46680553471182
iteration 2: 16.733448762295488
iteration 3: 4.646927113764621
iteration 4: 2.844103414066682
iteration 5: 2.6053624456990874
iteration 6: 2.517331912793828
iteration 7: 2.4511601920615775
iteration 8: 2.3860450643860514
iteration 9: 2.316310255003821
iteration 10: 2.23988278353822
iteration 11: 2.15585618425872
iteration 12: 2.063905620591499
iteration 13: 1.9641648988150853
iteration 14: 1.8572235975915405
iteration 15: 1.7441426647493223
iteration 16: 1.626446951614711
iteration 17: 1.5060721891133524
iteration 18: 1.3852558904242052
iteration 19: 1.2663749209711375
iteration 20: 1.151747165381156
iteration 21: 1.0434272749416866
iteration 22: 0.943032136683556
iteration 23: 0.8516272748326758
iteration 24: 0.7696915043134963
iteration 25: 0.6971585379492163
iteration 26: 0.6335176005864485
iteration 27: 0.5779457731360297
iteration 28: 0.5294444431526967
iteration 29: 0.48695881521067946
iterati

In [33]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.042604188027312

The actual value for the missing rating is nan

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [34]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
# fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
iteration 1: 23.420125275847088
iteration 2: 10.846599938082601
iteration 3: 7.437962501466384
iteration 4: 5.724298004453866
iteration 5: 4.642843888625154
iteration 6: 3.8876002673916656
iteration 7: 3.3271454874991084
iteration 8: 2.8941572193135983
iteration 9: 2.5502194793851327
iteration 10: 2.271507084788515
iteration 11: 2.042243356038841
iteration 12: 1.8514626522074005
iteration 13: 1.6912705213219557
iteration 14: 1.555815561496223
iteration 15: 1.440630929917063
iteration 16: 1.342199236927572
iteration 17: 1.2576710020684405
iteration 18: 1.1846914678721376
iteration 19: 1.1212981983398156
iteration 20: 1.0658582120823514


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [41]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = (~np.isnan(first_1000_users)).sum()
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = np.isnan(first_1000_users).sum()
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [42]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


In [43]:
preds

array([[ 3.55255907,  4.71568183,  2.67161184, ...,  2.61859979,
         2.70171128,  4.24753055],
       [ 3.56260202,  4.76007759,  2.74153218, ...,  2.80576542,
         2.43608181,  4.05926264],
       [ 2.94673296,  3.72333441,  1.97633806, ...,  1.90568837,
         2.44694401,  3.55100255],
       ..., 
       [ 2.64440393,  3.68409917,  2.1966166 , ...,  2.19720389,
         1.8122702 ,  3.12152358],
       [ 3.73880415,  4.7324726 ,  2.80500426, ...,  2.69403484,
         2.72070025,  4.33035716],
       [ 2.30057436,  3.20471382,  1.71434001, ...,  1.70748538,
         1.8748921 ,  2.87064761]])